<a href="https://colab.research.google.com/github/srikar999/TakeoutDataAnalytics/blob/main/Youtube_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [8]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd

In [9]:
spark = SparkSession.builder.master("local").appName("Test").getOrCreate()

In [ ]:
spark

# Channels Subscribed

In [117]:
subs=spark.read.option("inferSchema",True)\
.csv("/content/drive/MyDrive/Takeout_Project/input/Takeout/YouTube and YouTube Music/subscriptions/subscriptions.csv",header=True)
subs=subs.agg(F.countDistinct("Channel Id").alias("value"))
subs=subs.withColumn("load_dt",F.current_timestamp()).withColumn("fact",F.lit("Channels Subscribed"))
subs=subs.withColumn("year",F.year(F.col("load_dt"))).withColumn("month",F.month(F.col("load_dt")))
op_path="/content/drive/MyDrive/Takeout_Project/outputs/Youtube/subs_count/"
subs.coalesce(1).write.mode("overwrite").csv(op_path,header=True)

#Playlist data

In [118]:
import os
play_list_files="/content/drive/MyDrive/Takeout_Project/input/Takeout/YouTube and YouTube Music/playlists/"
df = pd.DataFrame()
df["year"]=[]
df["month"]=[]
df["Time Added_ts"]=[]
df["playlist"]=[]
df["count"]=[]
for i in os.listdir(play_list_files):
    playlist=pd.read_csv(play_list_files+i,skiprows=3)
    playlist["playlist"]=i.split(".")[0]
    playlist['Time Added_ts']=pd.to_datetime(playlist['Time Added'], format='%Y-%m-%d %H:%M:%S UTC')
    playlist["year"]=playlist["Time Added_ts"].dt.year
    playlist["month"]=playlist["Time Added_ts"].dt.month
    playlist["day"]=playlist["Time Added_ts"].dt.day
    #display(playlist)
    playlist=playlist[["Video Id","year","month","Time Added_ts","playlist"]].groupby(["year","month","Time Added_ts","playlist"]).count().reset_index()
    playlist["count"]=playlist["Video Id"]
    df=pd.concat([df,playlist[["year","month","Time Added_ts","playlist","count"]]])
df.to_csv("/content/drive/MyDrive/Takeout_Project/outputs/Youtube/Playlist_data.csv",)


In [119]:
for i in os.listdir(op_path):
  if "csv" in i:
    if "csv.crc" not in i:
      os.rename(op_path+i, op_path+"subs_count.csv")